# Libbitcoin SPV Client: Chain Header Validation

In this example, we demonstrate how to validate the proof of work of the header chain returned from the server.
<br>
<img src="images/spv_header_validation.jpg" alt="drawing" style="" width="800px"/>




### Libbitcoin-System (C++) Setup

In [5]:
// Compiler & linker information for c++ interpreter.
#pragma cling add_include_path("/usr/local/include","/usr/local/Cellar/zeromq/4.2.5/include","/usr/local/Cellar/libbitcoin-consensus/3.5.0/include")
#pragma cling add_library_path("/usr/local/lib","/usr/local/Cellar/zeromq/4.2.5/lib","/usr/local/Cellar/libbitcoin-consensus/3.5.0/lib")
#pragma cling load("bitcoin","bitcoin-protocol","bitcoin-blockchain","bitcoin-consensus","bitcoin-database","zmq","secp256k1","pthread","boost_chrono-mt","boost_date_time-mt","boost_filesystem","boost_iostreams-mt","boost_locale-mt","boost_log-mt","boost_program_options-mt","boost_regex-mt","boost_system","boost_thread-mt")

// Libbitcoin-System
// Libbitcoin-Protocol
// Libbitcoin-Blockchain
#include <bitcoin/bitcoin.hpp> 
#include <bitcoin/protocol.hpp> 
#include <bitcoin/blockchain.hpp> 

<hr style="border: 0.5px dashed #000;">


## 1) Setup ZMQ.


In [6]:
bc::protocol::zmq::context my_context(true);
bc::protocol::zmq::socket my_requester(my_context,
    bc::protocol::zmq::socket::role::requester);
bc::config::endpoint public_endpoint("tcp://mainnet2.libbitcoin.net:9091");

bc::code ec_socket; 
ec_socket = my_requester.connect(public_endpoint);


## 2) Initialise chain state at height 0

* `bc::config::settings` class contains chain parameters.
* `bc::chain::chain_state::data` is populated with `bc::config::settings` object.
* First `chain_state` is instantiated with populated`bc::chain::chain_state::data` object.


In [7]:
bc::config::settings my_settings(bc::config::settings::mainnet);
bc::blockchain::settings genesis_settings(my_settings);
auto genesis_block = bc::chain::block::genesis_mainnet();
bc::chain::chain_state::data genesis_data;
genesis_data.height = 0;
genesis_data.hash = bc::base16_literal(
    "000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f");
/// mainnet: 227931, testnet: 21111 (or map::unrequested)
genesis_data.allow_collisions_hash = bc::base16_literal(
    "000000000000024b89b42a942fe0d9fea3bb44ab7bd1b19115dd6a759c0808b8");
/// mainnet: 419328, testnet: 770112 (or map::unrequested)
genesis_data.bip9_bit0_hash = bc::base16_literal(
    "000000000000000004a1b34462cb8aeebd5799177f7a29cf28f2d1961716b5b5");
/// mainnet: 481824, testnet: 834624 (or map::unrequested)
genesis_data.bip9_bit1_hash = bc::base16_literal(
    "0000000000000000001c8018d9cb3b742ef25114f27563e3fc4a1902167f9893");
genesis_data.bits.self = genesis_block.header().bits();
genesis_data.version.self = genesis_block.header().version();
genesis_data.timestamp.self = genesis_block.header().version();

// Initialise genesis chain state.
//--------------------------------------------------------------------------
std::shared_ptr<bc::chain::chain_state> prev_chain_state_ptr(
    new bc::chain::chain_state(std::move(genesis_data),
    genesis_settings.checkpoints,
    genesis_settings.enabled_forks()
  ));

std::shared_ptr<bc::chain::chain_state> current_chain_state_ptr;

auto prev_block_hash = genesis_block.header().hash();


## 3) Request header at each height: Validate proof-of-work.
* *At each chain height:*
    * Request `header` from server.
    * Derive `chain_state` at height of header.
    * Validate `previous_hash` link of header.
    * Check whether header work is within target.

In [ ]:
// For this example: Validate until height 100000.
//--------------------------------------------------------------------------

for( uint32_t height = 1; height <= 100000; height += 1 )
{
  // Client request message.
    std::string command = "blockchain.fetch_block_header";
    uint32_t message_id(0);
    bc::protocol::zmq::message my_request;
    my_request.enqueue(bc::to_chunk(command));
    my_request.enqueue(bc::to_chunk(bc::to_little_endian(message_id)));
    my_request.enqueue(bc::to_chunk(bc::to_little_endian(height)));

    // Server response message.
    bc::protocol::zmq::message server_response;
    std::string my_message_command;
    uint32_t my_message_id;
    bc::data_chunk reply_payload;
    bc::code ec_reply;

    // Submit Query.
    ec_socket = my_request.send(my_requester);

    // Parse Reply.
    server_response.receive(my_requester);
    my_message_command = server_response.dequeue_text();
    server_response.dequeue(my_message_id);
    server_response.dequeue(reply_payload);
    bc::data_source reply_byte_stream(reply_payload);
    bc::istream_reader reply_byte_stream_reader(reply_byte_stream);
    ec_reply = reply_byte_stream_reader.read_error_code();

    bc::chain::header received_header;
    received_header.from_data(reply_byte_stream_reader);

    // Derive next chain_state.
    //----------------------------------------------------------------------
    if(prev_block_hash != received_header.previous_block_hash())
    {
        std::cout << "Broken hash link at height: " << height
            << "/n";
        return 0;
    }
    prev_block_hash = received_header.hash();

    current_chain_state_ptr.reset(new bc::chain::chain_state(
        *prev_chain_state_ptr, received_header));

    prev_chain_state_ptr = current_chain_state_ptr;

    // POW validation.
    //----------------------------------------------------------------------
    if(!received_header.is_valid_proof_of_work())
    {
        std::cout << "Header work/bits invalid at height: "
            << height << "/n";
        return 0;
    }

    // Validate required work from chain history.
    auto difficulty_bits = prev_chain_state_ptr->work_required();
    bc::uint256_t target(bc::chain::compact{difficulty_bits});

    auto work_performed = bc::to_uint256(received_header.hash());

    if(!(work_performed <= target))
    {
        std::cout << "Insufficient work: " << height << "/n";
        return 0;
    }

    // Print out at each height.
    bc::hash_digest header_hash = received_header.hash();
    std::reverse(header_hash.begin(), header_hash.end());

    std::stringstream target_string_stream;
    target_string_stream << std::setw(64) << std::setfill('0')
                         << std::hex << target;
    std::string target_string = target_string_stream.str();

    std::cout << std::dec << "height: \n"
              << height << "\n"
              << "header hash:\n"
              << bc::encode_base16(bc::to_chunk(header_hash)) << "\n"
              << "target:\n"
              << target_string
              << std::endl;

}

height: 
1
header hash:
00000000839a8e6886ab5951d76f411475428afc90947ee320161bbf18eb6048
target:
00000000FFFF0000000000000000000000000000000000000000000000000000
height: 
2
header hash:
000000006a625f06636b8bb6ac7b960a8d03705d1ace08b1a19da3fdcc99ddbd
target:
00000000FFFF0000000000000000000000000000000000000000000000000000
height: 
3
header hash:
0000000082b5015589a3fdf2d4baff403e6f0be035a5d9742c1cae6295464449
target:
00000000FFFF0000000000000000000000000000000000000000000000000000
height: 
4
header hash:
000000004ebadb55ee9096c9a2f8880e09da59c0d68b1c228da88e48844a1485
target:
00000000FFFF0000000000000000000000000000000000000000000000000000
height: 
5
header hash:
000000009b7262315dbf071787ad3656097b892abffd1f95a1a022f896f533fc
target:
00000000FFFF0000000000000000000000000000000000000000000000000000
height: 
6
header hash:
000000003031a0e73735690c5a1ff2a4be82553b2a12b776fbd3a215dc8f778d
target:
00000000FFFF0000000000000000000000000000000000000000000000000000
height: 
7
header hash:
0000